In [ ]:
import argparse
import os
import shutil
import cv2
import csv
import sys
from pathlib import Path

# --- Configuration ---
DEFAULT_THRESHOLD = 100.0
DEFAULT_OUTPUT_DIR = "./culled_blurry/"
IMAGE_EXTENSIONS = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff', '.webp']
# ---------------------

def is_blurry(image_path: str, threshold: float) -> tuple[bool, float]:
    """
    Calculates the blur score (Laplacian Variance) for a single image.

    The Laplacian operator highlights regions of rapid intensity change (edges).
    A high variance indicates a sharp image with many edges, while a low
    variance suggests a smooth, blurry image.

    Input:
        image_path: String path to the image file.
        threshold: The variance score below which an image is considered blurry.

    Output:
        A tuple: (is_blurry_flag: bool, variance_score: float).
    """
    try:
        # 1. Load the image using OpenCV
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Could not load image file: {image_path}. Skipping.")
            return False, 0.0

        # 2. Convert to grayscale
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # 3. Apply the Laplacian operator
        # cv2.CV_64F is used to store floating-point results, preventing overflow.
        laplacian = cv2.Laplacian(gray, cv2.CV_64F)

        # 4. Calculate the variance of the Laplacian result
        # The variance is the metric used to judge sharpness.
        _, variance_score = cv2.meanStdDev(laplacian)
        variance_score = variance_score[0][0] ** 2 # Variance is the standard deviation squared

        # 5. Compare the variance score to the threshold
        is_blurry_flag = variance_score < threshold

        return is_blurry_flag, variance_score

    except cv2.error as e:
        print(f"Error processing image {image_path} with OpenCV: {e}. Skipping.")
        return False, 0.0
    except Exception as e:
        print(f"An unexpected error occurred for {image_path}: {e}. Skipping.")
        return False, 0.0


def process_directory(input_dir: str, cull_dir: str, threshold: float, file_extensions: list) -> dict:
    """
    Scans the input directory, processes images, and moves the blurry ones.

    Input:
        input_dir: The directory containing the images to be scanned.
        cull_dir: The directory where blurry images will be moved.
        threshold: The blur sensitivity threshold.
        file_extensions: A list of file extensions to process (e.g., ['.jpg', '.png']).

    Output:
        A dictionary summarizing the operation and detailed results.
    """
    input_path = Path(input_dir)
    cull_path = Path(cull_dir)

    if not input_path.is_dir():
        print(f"Error: Input directory not found at '{input_dir}'")
        sys.exit(1)

    # 1. Ensure cull_dir exists; create it if not
    try:
        cull_path.mkdir(parents=True, exist_ok=True)
    except Exception as e:
        print(f"Error: Could not create output directory '{cull_dir}': {e}")
        sys.exit(1)

    results = []
    total_scanned = 0
    total_culled = 0

    print(f"\n--- Starting Scan ---\nInput Directory: {input_dir}\nOutput Directory: {cull_dir}\nThreshold: {threshold:.2f}\n")

    # 2. Recursively search input_dir for all files matching file_extensions
    for file_ext in file_extensions:
        # Use glob to find files matching extensions recursively
        for image_path in input_path.rglob(f"*{file_ext}"):
            if not image_path.is_file():
                continue

            total_scanned += 1
            original_path = str(image_path)
            relative_path = image_path.relative_to(input_path)

            # 3a. Call is_blurry() to get the score and flag
            is_blurry_flag, score = is_blurry(original_path, threshold)

            culled_path = None
            if is_blurry_flag:
                # 3b. If is_blurry_flag is True, move the image file
                total_culled += 1
                try:
                    # Preserve relative directory structure in the cull_dir
                    target_dir = cull_path / relative_path.parent
                    target_dir.mkdir(parents=True, exist_ok=True)
                    culled_path = target_dir / image_path.name

                    # Use shutil.move for speed
                    shutil.move(original_path, culled_path)
                    print(f"[CULLED] Score {score:.2f} < {threshold:.2f}: {relative_path}")
                except Exception as e:
                    print(f"Error moving file {original_path}: {e}")
                    # Revert status if move failed
                    is_blurry_flag = False
                    total_culled -= 1
                    culled_path = None
            else:
                print(f"[KEPT]   Score {score:.2f} >= {threshold:.2f}: {relative_path}")


            # 3c. Record the result
            results.append({
                'filename': str(relative_path),
                'score': score,
                'threshold': threshold,
                'culled': is_blurry_flag,
                'new_path': str(culled_path) if culled_path else None
            })

    # Output: A dictionary summarizing the operation
    return {
        'total_scanned': total_scanned,
        'total_culled': total_culled,
        'results': results
    }


def parse_arguments() -> argparse.Namespace:
    """
    Handles all command-line arguments.
    """
    parser = argparse.ArgumentParser(
        description="Blur-Be-Gone: An image utility to automatically cull blurry photos using the Laplacian Variance method."
    )

    # 1. -i, --input (Required: path to input_dir)
    parser.add_argument(
        '-i', '--input',
        type=str,
        required=True,
        help="Path to the input directory containing images to scan."
    )

    # 2. -t, --threshold (Optional: blur sensitivity; default should be 100.0)
    parser.add_argument(
        '-t', '--threshold',
        type=float,
        default=DEFAULT_THRESHOLD,
        help=f"The blur sensitivity threshold. Images with a variance score below this are considered blurry and culled. (Default: {DEFAULT_THRESHOLD:.1f})"
    )

    # 3. -o, --output (Optional: path to the cull_dir; default should be ./culled_blurry/)
    parser.add_argument(
        '-o', '--output',
        type=str,
        default=DEFAULT_OUTPUT_DIR,
        help=f"Path to the directory where blurry images will be moved (culled). (Default: {DEFAULT_OUTPUT_DIR})"
    )

    # 4. -l, --log-file (Optional: path to save the detailed results log)
    parser.add_argument(
        '-l', '--log-file',
        type=str,
        default=None,
        help="Optional path to save a detailed CSV log file of all scanned images and their scores."
    )

    return parser.parse_args()


def generate_report(results_dict: dict, log_path: str = None) -> None:
    """
    Writes the detailed and summarized results to the console and an optional file.

    Input:
        results_dict: The dictionary output from process_directory.
        log_path: Optional path to save the detailed results log (CSV format).
    """
    total_scanned = results_dict['total_scanned']
    total_culled = results_dict['total_culled']

    # 1. Print a summary to the console
    print("\n--- Summary Report ---")
    print(f"Total images scanned: {total_scanned}")
    print(f"Total blurry images culled: {total_culled}")
    print(f"Total images remaining: {total_scanned - total_culled}")

    # 2. If log_path is provided, write the detailed results to a CSV file
    if log_path:
        log_path = Path(log_path)
        print(f"\nWriting detailed results to log file: {log_path}")

        fieldnames = ['filename', 'score', 'threshold', 'culled', 'new_path']
        try:
            with open(log_path, 'w', newline='', encoding='utf-8') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for result in results_dict['results']:
                    writer.writerow(result)
            print("Log file successfully saved.")
        except Exception as e:
            print(f"Error writing log file to {log_path}: {e}")
            print("Detailed report will not be saved.")


def main():
    """Main execution function to run the Blur-Be-Gone utility."""
    print("--- Blur-Be-Gone Image Culling Utility ---")

    # 1. Parse Arguments
    args = parse_arguments()

    # Define the image extensions we are looking for
    extensions = [e.lower() for e in IMAGE_EXTENSIONS]

    # 2. Process Directory
    # Note: We pass the arguments directly to the processing function.
    results = process_directory(
        input_dir=args.input,
        cull_dir=args.output,
        threshold=args.threshold,
        file_extensions=extensions
    )

    # 3. Generate Report
    generate_report(results, args.log_file)
    print("\n--- Operation Complete. Have a sharp day! ---")

if __name__ == "__main__":
    # Ensure OpenCV is available for the core algorithm
    if 'cv2' not in sys.modules:
        print("Error: The 'opencv-python' library (cv2) is required but not found.")
        print("Please install it: pip install opencv-python")
    else:
        main()